In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from matplotlib import pyplot as plt
from My_Basic_Net.utils.Dataprocess import Loader
from My_Basic_Net.utils.OP import Bn_bin_conv_pool
from My_Basic_Net.utils.plot import plot_cfm
import numpy as np
import random


batch_size = 64
classes_num = 17
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

seed = 110
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
set_seed(seed)

loader = Loader(batch_size=batch_size, classes_num=classes_num, device=device, test_size=0.2)
labels, train_loader, test_loader = loader.loader()

lr = 0.01
epochs = 1000

########################################定义网络##############################################
kernel_size, padding, poolsize =11, 5, 5
class ECG_Bin(nn.Module):
    def __init__(self, device):
        super(ECG_Bin, self).__init__()
        self.name = 'Bin_ECG'
        self.device = device
        self.classifier = nn.Sequential(
            # input_channels, output_channels, kernel_size:9, stride, padding:7, pad_value, pool_size:5, pool_stride
            Bn_bin_conv_pool(1, 8, kernel_size, 3, padding, 1,
                             poolsize, 2),
            Bn_bin_conv_pool(8, 16, kernel_size, 1, padding, 1,
                             poolsize, 2),
            Bn_bin_conv_pool(16, 32, 9, 1, padding, 1,
                             poolsize, 2),
            Bn_bin_conv_pool(32, 64, 9, 1, padding, 1,
                             poolsize, 2),# +
            Bn_bin_conv_pool(64, 32, kernel_size, 1, padding, 1,
                             poolsize, 2),
            Bn_bin_conv_pool(32, classes_num, kernel_size, 1, padding, 1,
                             poolsize, 2)
        )
        self.dropout = nn.Dropout(p=0.5)  # 防止过拟合

    def forward(self, batch_data):
        batch_data = batch_data.clone().detach().requires_grad_(True).to(self.device)
        batch_data = self.classifier(batch_data)

        batch_data = self.dropout(batch_data)
        batch_data = batch_data.mean(dim=2)  # 去掉一个维度

        return batch_data

model = ECG_Bin(device=device).to(device)

#####################################损失函数：交叉熵#########################################
loss_fn = nn.CrossEntropyLoss().to(device)

########################################优化器##############################################
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = optim.Adam(model.parameters(), lr=lr)
# optimizer = optim.Adam(model.parameters(), lr=0.1)
# # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

print(device)
#################################学习率梯度衰减#################################
# 1. 带预热的余弦退火
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, epochs, 1, 0.02)
# 2. 余弦退火：T_max指一次学习率周期的迭代次数，即T_max个epoch后重置学习率。eta_min指最小学习率，即一个周期后学习率最小会下降到的值
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0.05,last_epoch=-1)
# 3. 指数衰减,每一次epoch，lr都乘gamma
# torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma= 0.5)

####################################模型参数汇总#######################################
from torchinfo import summary

summary(model=model,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])



X shape before reshape: torch.Size([1024, 3600])
cuda:0


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ECG_Bin (ECG_Bin)                        [64, 1, 3600]        [64, 17]             --                   True
├─Sequential (classifier)                [64, 1, 3600]        [64, 17, 16]         --                   True
│    └─Bn_bin_conv_pool (0)              [64, 1, 3600]        [64, 8, 598]         --                   True
│    │    └─ConstantPad1d (pad)          [64, 1, 3600]        [64, 1, 3610]        --                   --
│    │    └─BinaryConv1d (conv)          [64, 1, 3610]        [64, 8, 1200]        88                   True
│    │    └─MaxPool1d (pool)             [64, 8, 1200]        [64, 8, 598]         --                   --
│    │    └─PReLU (prelu)                [64, 8, 598]         [64, 8, 598]         1                    True
│    │    └─BatchNorm1d (bn)             [64, 8, 598]         [64, 8, 598]         16                   True
│    └─Bn_bin_conv